In [1]:
import pandas as pd
import numpy as np
import yaml
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.float_format = '{:.6f}'.format
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import json
from typing import List
import math

plt.rcParams["lines.linewidth"] = 2
plt.rcParams["axes.labelsize"] = 18
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["legend.fontsize"] = 15
plt.rcParams["xtick.labelsize"] = 14
plt.rcParams["ytick.labelsize"] = 14
import os
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
import tqdm

In [2]:
import altair as alt
from timeshap.plot import timeshap_theme

alt.themes.register("timeshap_theme", timeshap_theme)
alt.themes.enable("timeshap_theme")

ThemeRegistry.enable('timeshap_theme')

# Set random seeds

In [3]:
np.random.seed(42)

# Load the data

In [4]:
data_directories = next(os.walk("./AReM"))[1]

In [5]:
all_csvs = []
for folder in data_directories:
    if folder in ['bending1', 'bending2']:
        continue
    folder_csvs = next(os.walk(f"./AReM/{folder}"))[2]
    for data_csv in folder_csvs:
        if data_csv == 'dataset8.csv':
            # this dataset only has 479 instances
            # it is possible to use it, but would require padding logic
            continue
        loaded_data = pd.read_csv(f"./AReM/{folder}/{data_csv}", skiprows=4)
        print(f"{folder}/{data_csv} ------ {loaded_data.shape}")
        
        csv_id = re.findall(r'\d+', data_csv)[0]
        loaded_data['id'] = csv_id
        loaded_data['all_id'] = f"{folder}_{csv_id}"
        loaded_data['activity'] = folder
        all_csvs.append(loaded_data)
all_data = pd.concat(all_csvs)

sitting/dataset5.csv ------ (480, 7)
sitting/dataset11.csv ------ (480, 7)
sitting/dataset3.csv ------ (480, 7)
sitting/dataset10.csv ------ (480, 7)
sitting/dataset12.csv ------ (480, 7)
sitting/dataset4.csv ------ (480, 7)
sitting/dataset13.csv ------ (480, 7)
sitting/dataset9.csv ------ (480, 7)
sitting/dataset2.csv ------ (480, 7)
sitting/dataset15.csv ------ (480, 7)
sitting/dataset1.csv ------ (480, 7)
sitting/dataset14.csv ------ (480, 7)
sitting/dataset6.csv ------ (480, 7)
sitting/dataset7.csv ------ (480, 7)
standing/dataset5.csv ------ (480, 7)
standing/dataset11.csv ------ (480, 7)
standing/dataset3.csv ------ (480, 7)
standing/dataset10.csv ------ (480, 7)
standing/dataset12.csv ------ (480, 7)
standing/dataset4.csv ------ (480, 7)
standing/dataset13.csv ------ (480, 7)
standing/dataset9.csv ------ (480, 7)
standing/dataset2.csv ------ (480, 7)
standing/dataset15.csv ------ (480, 7)
standing/dataset1.csv ------ (480, 7)
standing/dataset14.csv ------ (480, 7)
standing/datas

In [6]:
all_data.columns = ['timestamp', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23', 'id', 'all_id', 'activity']

### Separate in train and test

In [7]:
# all ids ['1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6', '7', '8', '9']
#choose ids to use for test
ids_for_test = np.random.choice(['1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6', '7', '8', '9'],size = 4, replace=False)

In [8]:
d_train =  all_data[~all_data['id'].isin(ids_for_test)]
d_test = all_data[all_data['id'].isin(ids_for_test)]

### Model Features

In [9]:
raw_model_features = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']

### Normalize Features

In [10]:
class NumericalNormalizer:
    def __init__(self, fields: List[str]):
        self.metrics = {}
        self.fields = fields

    def fit(self, df: pd.DataFrame ) -> List[str]:
        means = df.mean()
        std = df.std()
        for field in self.fields:
            field_mean = means[field]
            field_stddev = std[field]
            self.metrics[field] = {'mean': field_mean, 'std': field_stddev}

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        # Transform to zero-mean and unit variance.
        for field in self.fields:
            f_mean = self.metrics[field]['mean']
            f_stddev = self.metrics[field]['std']
            # OUTLIER CLIPPING to [avg-3*std, avg+3*avg]
            df[field] = df[field].apply(lambda x: f_mean - 3 * f_stddev if x < f_mean - 3 * f_stddev else x)
            df[field] = df[field].apply(lambda x: f_mean + 3 * f_stddev if x > f_mean + 3 * f_stddev else x)
            if f_stddev > 1e-5:
                df[f'p_{field}_normalized'] = df[field].apply(lambda x: ((x - f_mean)/f_stddev))
            else:
                df[f'p_{field}_normalized'] = df[field].apply(lambda x: x * 0)
        return df

In [11]:
normalizor = NumericalNormalizer(raw_model_features)
normalizor.fit(d_train)
d_train_normalized = normalizor.transform(d_train)
d_test_normalized = normalizor.transform(d_test)

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try us

# Features

In [13]:
model_features = [f"p_{x}_normalized" for x in raw_model_features]
time_feat = 'timestamp'
label_feat = 'activity'
sequence_id_feat = 'all_id'

plot_feats = {
    'p_avg_rss12_normalized': "Mean Chest <-> Right Ankle",
    'p_var_rss12_normalized': "STD Chest <-> Right Ankle",
    'p_avg_rss13_normalized': "Mean Chest <-> Left Ankle",
    'p_var_rss13_normalized': "STD Chest <-> Left Ankle",
    'p_avg_rss23_normalized': "Mean Right Ankle <-> Left Ankle",
    'p_var_rss23_normalized': "STD Right Ankle <-> Left Ankle",
}

# Transform the dataset from multi-label to binary classification

### Select the activity to predict

In [14]:
# possible activities ['cycling', 'lying', 'sitting', 'standing', 'walking']
chosen_activity = 'cycling'

d_train_normalized['label'] = d_train_normalized['activity'].apply(lambda x: int(x == chosen_activity))
d_test_normalized['label'] = d_test_normalized['activity'].apply(lambda x: int(x == chosen_activity))


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
def df_to_numpy(df, model_feats, label_feat, group_by_feat, timestamp_Feat):
    sequence_length = len(df[timestamp_Feat].unique())

    data_tensor = np.zeros(
        (len(df[group_by_feat].unique()), sequence_length, len(model_feats)))
    labels_tensor = np.zeros((len(df[group_by_feat].unique()), 1))

    for i, name in enumerate(df[group_by_feat].unique()):
        name_data = df[df[group_by_feat] == name]
        sorted_data = name_data.sort_values(timestamp_Feat)

        data_x = sorted_data[model_feats].values
        labels = sorted_data[label_feat].values
        assert labels.sum() == 0 or labels.sum() == len(labels)
        data_tensor[i, :, :] = data_x
        labels_tensor[i, :] = labels[0]
    return data_tensor, labels_tensor


In [16]:
X_train, y_train = df_to_numpy(d_train_normalized, model_features, 'label', sequence_id_feat, time_feat)

X_test, y_test = df_to_numpy(d_test_normalized, model_features, 'label', sequence_id_feat, time_feat)

# Define and train Model

In [43]:
import tensorflow as tf

inputs = tf.keras.layers.Input(shape=(None, 6))
lstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(inputs)
ff1 = tf.keras.layers.Dense(64, activation='relu')(lstm1)
ff2 = tf.keras.layers.Dense(1, activation='sigmoid')(ff1)
model = tf.keras.models.Model(inputs=inputs, outputs=ff2)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.001))

model.fit(X_train, y_train, epochs=20, batch_size=55, validation_data=(X_test, y_test))

Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 0.6880 - val_loss: 0.6628
Epoch 2/20
1/1 [==============================] - 0s 263ms/step - loss: 0.6443 - val_loss: 0.6282
Epoch 3/20
1/1 [==============================] - 0s 275ms/step - loss: 0.6040 - val_loss: 0.5956
Epoch 4/20
1/1 [==============================] - 0s 260ms/step - loss: 0.5660 - val_loss: 0.5640
Epoch 5/20
1/1 [==============================] - 0s 261ms/step - loss: 0.5301 - val_loss: 0.5332
Epoch 6/20
1/1 [==============================] - 0s 265ms/step - loss: 0.4959 - val_loss: 0.5038
Epoch 7/20
1/1 [==============================] - 0s 270ms/step - loss: 0.4630 - val_loss: 0.4755
Epoch 8/20
1/1 [==============================] - 0s 263ms/step - loss: 0.4313 - val_loss: 0.4477
Epoch 9/20
1/1 [==============================] - 0s 300ms/step - loss: 0.4008 - val_loss: 0.4202
Epoch 10/20
1/1 [==============================] - 0s 274ms/step - loss: 0.3711 - val_loss: 0.3931
Epoch 11/20
1/1 [=====

# TimeSHAP


In [44]:
import timeshap

# Model entry point

In [45]:
f = lambda x: model.predict(x)

### Baseline event

In [46]:
from timeshap import calc_avg_event
average_event = calc_avg_event(d_train_normalized, numerical_feats=model_features, categorical_feats=[])

### Average score over baseline

In [47]:
from timeshap import get_avg_score_with_avg_event
avg_score_over_len = get_avg_score_with_avg_event(f, average_event, top=480)

# Global Explanations

### Explain all 

TimeSHAP offers methods to explain all instances and save as CSV.
This allows for global explanations and local plots with no calculation delay.

In [48]:
from timeshap import global_report
pos_dataset = d_test_normalized[d_test_normalized['label'] == 1]
pruning_dict = {'tol': [0.05, 0.075, 0.076], 'path': 'prun_all_tf.csv'}
event_dict = {'path': 'event_all_tf.csv', 'rs': 42, 'nsamples': 32000}
feature_dict = {'path': 'feature_all_tf.csv', 'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats,}
prun_stats, global_plot = global_report(f, pos_dataset, pruning_dict, event_dict, feature_dict, sequence_id_feat, time_feat, model_features, baseline=average_event)
prun_stats

Calculating pruning algorithms
Calculating pruning indexes
Calculating event data
No path to event data provided. Calculating data
No path to event data provided. Calculating data
No path to event data provided. Calculating data
No path to event data provided. Calculating data
No path to event data provided. Calculating data
No path to event data provided. Calculating data
No path to event data provided. Calculating data
No path to event data provided. Calculating data
No path to event data provided. Calculating data
Calculating global event plot
Calculating feat data
No path to feature data provided. Calculating data
No path to feature data provided. Calculating data
No path to feature data provided. Calculating data
No path to feature data provided. Calculating data
No path to feature data provided. Calculating data
No path to feature data provided. Calculating data
No path to feature data provided. Calculating data
No path to feature data provided. Calculating data
No path to featur

,Tolerance,Mean,Std
0,0.050000,473.000000,1.732051
1,0.075000,471.666667,1.154701
2,0.076000,471.666667,1.154701
3,No Pruning,480.000000,0.000000


In [49]:
global_plot

alt.HConcatChart(...)

### Select sequences to explain

##### 1 positive sequence and 1 negative sequence

In [50]:
positive_sequence_id = 'cycling_1'
pos_x_pd = d_test_normalized[d_test_normalized['all_id'] == positive_sequence_id]

# select model features only
pos_x_data = pos_x_pd[model_features]
# convert the instance to numpy so TimeSHAP receives it
pos_x_data = np.expand_dims(pos_x_data.to_numpy().copy(), axis=0)

### Local Report on positive instance

In [51]:
from timeshap.methods import local_report
pruning_dict = {'tol': 0.025}
event_dict = {'rs': 42, 'nsamples': 32000}
feature_dict = {'path': 'feature.csv', 'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats}
cell_dict_2 = {'rs': 42, 'nsamples': 32000, 'top_x_events': 2, 'top_x_feats': 2}
local_report(f, pos_x_data, pruning_dict, event_dict, feature_dict, cell_dict=cell_dict_2, entity_uuid=positive_sequence_id, entity_col='all_id', baseline=average_event)

No path to methods data provided. Calculating data
No path to event data provided. Calculating data
No path to feature data provided. Calculating data


alt.HConcatChart(...)

## Individual Plots

### Global Plots

In [52]:
from timeshap.methods import prune_all, pruning_statistics, event_explain_all, feat_explain_all
from timeshap.plot import plot_global_event, plot_global_feat

pos_dataset = d_test_normalized[d_test_normalized['label'] == 1]

In [53]:
pruning_dict = {'tol': [0.05, 0.075, 0.076], 'path': 'prun_all_tf.csv'}
prun_indexes = prune_all(f, pos_dataset, sequence_id_feat, average_event, pruning_dict, model_features, time_feat)
pruning_stats = pruning_statistics(prun_indexes, pruning_dict.get('tol'), sequence_id_feat)
pruning_stats

,Tolerance,Mean,Std
0,0.050000,473.000000,1.732051
1,0.075000,471.666667,1.154701
2,0.076000,471.666667,1.154701
3,No Pruning,480.000000,0.000000


In [54]:
event_dict = {'path': 'event_all_tf.csv', 'rs': 42, 'nsamples': 32000}
event_data = event_explain_all(f, pos_dataset, sequence_id_feat, average_event, event_dict, prun_indexes, model_features, time_feat)
event_global_plot = plot_global_event(event_data)
event_global_plot

alt.Chart(...)

In [55]:
feature_dict = {'path': 'feature_all_tf.csv', 'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats, }
feat_data = feat_explain_all(f, pos_dataset, sequence_id_feat, average_event, feature_dict, prun_indexes, model_features, time_feat)
feat_global_plot = plot_global_feat(feat_data, **feature_dict)
feat_global_plot

alt.Chart(...)

### Local Plots

In [56]:
from timeshap.plot import plot_temp_coalition_pruning, plot_event_heatmap, plot_feat_barplot, plot_cell_level
from timeshap.methods import local_pruning, local_event, local_feat, local_cell_level
# select model features only
pos_x_data = pos_x_pd[model_features]
# convert the instance to numpy so TimeSHAP receives it
pos_x_data = np.expand_dims(pos_x_data.to_numpy().copy(), axis=0)

In [58]:
pruning_dict = {'tol': 0.025,}
coal_plot_data, coal_prun_idx = local_pruning(f, pos_x_data, pruning_dict, average_event, positive_sequence_id, sequence_id_feat, False)
# coal_prun_idx is in negative terms
pruning_idx = pos_x_data.shape[1] + coal_prun_idx
pruning_plot = plot_temp_coalition_pruning(coal_plot_data, coal_prun_idx, 40)
pruning_plot

No path to methods data provided. Calculating data


alt.LayerChart(...)

In [59]:
event_dict = {'rs': 42, 'nsamples': 32000}
event_data = local_event(f, pos_x_data, event_dict, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
event_plot = plot_event_heatmap(event_data)
event_plot

No path to event data provided. Calculating data


alt.LayerChart(...)

In [60]:
feature_dict = {'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats}
feature_data = local_feat(f, pos_x_data, feature_dict, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
feature_plot = plot_feat_barplot(feature_data, feature_dict.get('top_feats'), feature_dict.get('plot_features'))
feature_plot

No path to feature data provided. Calculating data


alt.LayerChart(...)

In [61]:
cell_dict = {'rs': 42, 'nsamples': 32000, 'top_x_events': 3, 'top_x_feats': 3}
cell_data = local_cell_level(f, pos_x_data, cell_dict, event_data, feature_data, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
feat_names = list(feature_data['Feature'].values)[:-1] # exclude pruned events
cell_plot = plot_cell_level(cell_data, feat_names, feature_dict.get('plot_features'))
cell_plot

No path to feature data provided. Calculating data


alt.HConcatChart(...)